# Fases de entrenamiento de una red neuronal

Importamos los datos de un corpus en español.

Corpus de español:
- AnCora | Github: https://github.com/UniversalDependencies/UD_Spanish-AnCora

- usamos el conllu parser para leer el corpus: https://pypi.org/project/conllu/ . Es muy usado, se puede trabajar con pip install conllu

- Etiquetas Universal POS (Documentación): https://universaldependencies.org/u/pos/ . Hay referencias universales para clasificar palabras


In [1]:
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
fatal: destination path 'UD_Spanish-AnCora' already exists and is not an empty directory.


In [29]:
#leer los datos que vienen de un archivo conllu
from conllu import parse_incr
wordlist = []#creo una lista vacía
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") #abro el archivo que descargué al clonar el repositorio, agrego permisos de lectura con "r"

#for tokenlist in parse_incr(data_file):
#    print(tokenlist.serialize())
    
    #quiero ver los tokens
    #serialize es un atributo propio del formato conllu, sirve para poder ver bien los tokens


Veamos la estructura de los tokens

In [3]:
print(tokenlist)
print(tokenlist[1])

TokenList<Lo, cierto, es, que, a, mí, ,, me, da, un, poco, de, pena, .>
cierto


In [4]:
tokenlist[1]['form']+'|'+tokenlist[1]['upos']# el atributo form es una palabra, upos es la categoría gramatical.

'cierto|ADJ'

In [5]:
for i in range(10):
    print(tokenlist[i]['form']+'|'+tokenlist[i]['upos'])

Lo|PRON
cierto|ADJ
es|AUX
que|SCONJ
a|ADP
mí|PRON
,|PUNCT
me|PRON
da|VERB
un|DET


# Entrenamiento del modelo- Calculo de conteos
La primera etapa es el cálculo de conteos

- tags(tags) ``tagCountDict``: $C(tag)$
- emisiones(word|tag) ``emissionProbDict``: $C(word|tag)$
- transiciones(tag|prevtag) ``transitionDict``: $C(tag|prevtag)$

In [23]:
#tenemos tres diccionarios inicialmente vacíos

tagCountDict = {}
emissionDict = {}
transitionDict = {}

tagtype = 'upos'
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") #abro el archivo que descargué al clonar el repositorio, agrego permisos de lectura con "r"
for tokenlist in parse_incr(data_file):
    prevtag=None
    for token in tokenlist:
        #C(tag) el conteo de las etiquetas
        tag = token[tagtype]
        if tag in tagCountDict.keys():
            tagCountDict[tag] +=1
        else:
            tagCountDict[tag] = 1

        #C(WORD|tag) -> probabilidades de emisión
        wordtag = token['form'].lower()+'|'+token[tagtype] #(palabra|etiqueta)
        if wordtag in emissionProbDict.keys():
            emissionDict[wordtag] +=1
        else:
            emissionDict[wordtag] = 1

        #Ahora vamos a hacer un conteo para las probabilidades de transición
        #C(tag|tag_previo)
        
        if prevtag is None:
            prevtag=tag
            continue
        transitiontags = tag+'|'+prevtag
        if transitiontags in transitionDict.keys():
            transitionDict[transitiontags] = transitionDict[transitiontags] + 1
        else:
            transitionDict[transitiontags] = 1
        prevtag = tag    


# Cálculo de probabilidades

- Probabilidades de transición
$$
P(tag|prevtag)=\frac{C(prevtag,tag)}{C(prevtag)}
$$


- Probabilidades de emisión

$$
P(word|tag)=\frac{C(word,tag)}{C(tag)}
$$

In [28]:
transitionProbDict = {} # matriz A
emissionProbDict = {} # matriz B

# transition Probabilities 
for key in transitionDict.keys():
    tag, prevtag = key.split('|')
    if tagCountDict[prevtag]>0:
        transitionProbDict[key] = transitionDict[key]/(tagCountDict[prevtag])
    else:
        print(key)

# emission Probabilities 
for key in emissionDict.keys():
    word, tag = key.split('|')
    if emissionDict[key]>0:
        emissionProbDict[key] = emissionDict[key]/tagCountDict[tag]
    else:
        print(key)




In [27]:
#Ya tenemos el modelo, ahora tenemos que guardarlo
import numpy as np
np.save('transitionHMM.npy', transitionProbDict)
np.save('emissionHMM.npy', emissionProbDict)
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
transitionProbDict['ADJ|ADJ']

0.030225988700564973

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=31a57299-7db0-417b-9a80-6e17fdb92497' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>